# Motivation
There is little work on pushing active learning to latent space in recommendation tasks. The goal of this project is to show how deep recommender systems can actively learn from sequential user feedback and recommend personalized items to users.

# Data Preprocess
The dataset used was MovieLens 1M which was not uploaded to Github as Github doesn't allow it. Therefore, to run, you need to first download and move it under a directory called data. You can find data split example commands below.

In [ ]:
python3 getmovielens.py --implicit
python3 getmovielens.py --implicit --disable-validation

# Related Work

Thompson Sampling (Chapelle et al. 2011) draws samples from posterior distribution every iteration and chooses items that maximizing the expected reward. Then update posterior distribution based on observations.

UCB (Auer. 2002) chooses items that maximizing the expected reward plus an upper bound. 

Greedy algorithm always chooses items of the highest expected reward.

VAE-CF (Liang et al. 2018) is our core deep recommender system. It extends variational autoencoders to include collaborative filtering for implicit feedback, and models user-item implicit feedback using multinomial conditional likelihood. 

In Figure below, ${x}$ represents user click history, $\phi$ is the encoder parameter, $\theta$ is the decoder parameter, ${\epsilon}$ is a standard Gaussian distribution and ${z}$ represents the latent user representation.

<img src="VAECF.png">

# Experiment
First of all, we will split the dataset into train, active and test set based on timestamp. After we train VAE-CF (Liang et al. 2018) on the training set, we will choose the top-k items from either items received positive ratings or all items in the active set using active learning methods such as Greedy algorithm, Thompson sampling and UCB. Finally, we will update VAE-CF with the top-k items selected from the active set (k=50 in our experiments) and evaluate each algorithm on the test set using Precision@K, Recall@K and mAP@K, etc. Because we don't directly evaluate performance during active learning phase, we will also compare the results of choosing top-k items iteratively or one-shot from the active set.

# Research Question 1: Is active learning helping to boost recommendation general performance?

Table 1. Iterative. Sample from all data

In [1]:
import pandas as pd
df = pd.read_csv('final_result.csv', sep='\t')
df[(df['Iterative'] == True) & (df['SampleFromAll'] == True)].groupby(['Model', 'C'], as_index=False)['MAP@5', 'MAP@10', 'MAP@15', 'MAP@20', 'MAP@50', 'NDCG',
'Precision@5', 'Precision@10', 'Precision@15', 'Precision@20', 'Precision@50', 'R-Precision', 'Recall@5',
'Recall@10', 'Recall@15', 'Recall@20', 'Recall@50'].agg({'MAP@5': 'mean', 'MAP@10': 'mean', 'MAP@15': 'mean', 
'MAP@20': 'mean', 'MAP@50': 'mean', 'NDCG': 'mean', 'Precision@5': 'mean', 'Precision@10': 'mean', 
'Precision@15': 'mean', 'Precision@20': 'mean', 'Precision@50': 'mean', 'R-Precision': 'mean', 'Recall@5': 'mean',
'Recall@10': 'mean', 'Recall@15': 'mean', 'Recall@20': 'mean', 'Recall@50': 'mean'})

,Model,C,MAP@5,MAP@10,MAP@15,MAP@20,MAP@50,NDCG,Precision@5,Precision@10,Precision@15,Precision@20,Precision@50,R-Precision,Recall@5,Recall@10,Recall@15,Recall@20,Recall@50
0,Greedy,0.0,0.567882,0.525302,0.491568,0.463978,0.360013,0.233142,0.525827,0.456693,0.403990,0.367224,0.241811,0.169505,0.037910,0.065046,0.085826,0.103835,0.171665
1,ThompsonSampling,0.0,0.561318,0.520612,0.487964,0.460984,0.357428,0.230464,0.521890,0.453512,0.403633,0.366031,0.238929,0.167140,0.037526,0.064626,0.085784,0.103524,0.169395
2,UCB,0.5,0.568339,0.525290,0.491442,0.464425,0.359649,0.231867,0.531654,0.457402,0.405039,0.369173,0.239748,0.168059,0.038300,0.065245,0.086142,0.104449,0.170173
3,UCB,1.0,0.569887,0.526123,0.492442,0.465019,0.360414,0.233027,0.526677,0.455874,0.406341,0.368504,0.241493,0.169060,0.037898,0.064948,0.086318,0.104304,0.171330
4,UCB,2.0,0.573098,0.528425,0.493979,0.466115,0.360171,0.232875,0.531528,0.457291,0.405879,0.367559,0.240970,0.168617,0.038263,0.065235,0.086196,0.103964,0.170792


Table 2. Iterative. Sample from only positive data

In [2]:
import pandas as pd
df = pd.read_csv('final_result.csv', sep='\t')
df[(df['Iterative'] == True) & (df['SampleFromAll'] == False)].groupby(['Model', 'C'], as_index=False)['MAP@5', 'MAP@10', 'MAP@15', 'MAP@20', 'MAP@50', 'NDCG',
'Precision@5', 'Precision@10', 'Precision@15', 'Precision@20', 'Precision@50', 'R-Precision', 'Recall@5',
'Recall@10', 'Recall@15', 'Recall@20', 'Recall@50'].agg({'MAP@5': 'mean', 'MAP@10': 'mean', 'MAP@15': 'mean', 
'MAP@20': 'mean', 'MAP@50': 'mean', 'NDCG': 'mean', 'Precision@5': 'mean', 'Precision@10': 'mean', 
'Precision@15': 'mean', 'Precision@20': 'mean', 'Precision@50': 'mean', 'R-Precision': 'mean', 'Recall@5': 'mean',
'Recall@10': 'mean', 'Recall@15': 'mean', 'Recall@20': 'mean', 'Recall@50': 'mean'})

,Model,C,MAP@5,MAP@10,MAP@15,MAP@20,MAP@50,NDCG,Precision@5,Precision@10,Precision@15,Precision@20,Precision@50,R-Precision,Recall@5,Recall@10,Recall@15,Recall@20,Recall@50
0,Greedy,0.0,0.625411,0.584062,0.555060,0.531863,0.440809,0.301108,0.580913,0.522677,0.482016,0.449843,0.329398,0.233702,0.042423,0.075868,0.104571,0.130009,0.237275
1,ThompsonSampling,0.0,0.621474,0.582742,0.554001,0.530491,0.439252,0.299699,0.582173,0.522268,0.479895,0.447606,0.327843,0.232349,0.042478,0.075856,0.104130,0.129169,0.236139
2,UCB,0.5,0.631832,0.589725,0.560081,0.535568,0.440719,0.300095,0.588819,0.527874,0.484199,0.449449,0.326047,0.231266,0.043138,0.076818,0.105220,0.130043,0.234936
3,UCB,1.0,0.624056,0.583961,0.555210,0.531770,0.440146,0.300145,0.581984,0.522803,0.481491,0.449181,0.327761,0.232511,0.042550,0.075978,0.104674,0.129830,0.236125
4,UCB,2.0,0.625411,0.585460,0.556253,0.532480,0.440413,0.299984,0.582488,0.524126,0.482058,0.448583,0.327515,0.232289,0.042706,0.076199,0.104748,0.129541,0.235747


Table 3. Non-Iterative. Sample from all data

In [3]:
import pandas as pd
df = pd.read_csv('final_result.csv', sep='\t')
df[(df['Iterative'] == False) & (df['SampleFromAll'] == True)].groupby(['Model', 'C'], as_index=False)['MAP@5', 'MAP@10', 'MAP@15', 'MAP@20', 'MAP@50', 'NDCG',
'Precision@5', 'Precision@10', 'Precision@15', 'Precision@20', 'Precision@50', 'R-Precision', 'Recall@5',
'Recall@10', 'Recall@15', 'Recall@20', 'Recall@50'].agg({'MAP@5': 'mean', 'MAP@10': 'mean', 'MAP@15': 'mean', 
'MAP@20': 'mean', 'MAP@50': 'mean', 'NDCG': 'mean', 'Precision@5': 'mean', 'Precision@10': 'mean', 
'Precision@15': 'mean', 'Precision@20': 'mean', 'Precision@50': 'mean', 'R-Precision': 'mean', 'Recall@5': 'mean',
'Recall@10': 'mean', 'Recall@15': 'mean', 'Recall@20': 'mean', 'Recall@50': 'mean'})

,Model,C,MAP@5,MAP@10,MAP@15,MAP@20,MAP@50,NDCG,Precision@5,Precision@10,Precision@15,Precision@20,Precision@50,R-Precision,Recall@5,Recall@10,Recall@15,Recall@20,Recall@50
0,Greedy,0.0,0.559470,0.516241,0.483123,0.455906,0.349279,0.224324,0.519055,0.446677,0.398068,0.358630,0.230961,0.160961,0.037405,0.063629,0.084639,0.101207,0.163064
1,ThompsonSampling,0.0,0.518442,0.474580,0.442955,0.418123,0.329114,0.217527,0.470425,0.407276,0.363066,0.331449,0.230362,0.160909,0.033785,0.057846,0.076937,0.093807,0.163376
2,UCB,0.5,0.548774,0.502971,0.468229,0.440940,0.341609,0.222113,0.499843,0.430551,0.381365,0.345276,0.230535,0.161268,0.035883,0.061287,0.080923,0.097527,0.163510
3,UCB,1.0,0.541444,0.498199,0.465248,0.438419,0.339709,0.221170,0.498898,0.430394,0.380997,0.344110,0.230431,0.161174,0.035883,0.061284,0.080895,0.097398,0.163288
4,UCB,2.0,0.531462,0.484496,0.450016,0.423740,0.331025,0.218124,0.482457,0.411055,0.364556,0.332441,0.229109,0.160291,0.034507,0.058493,0.077485,0.094213,0.162616


Table 4. Non-Iterative. Sample from only positive data

In [4]:
import pandas as pd
df = pd.read_csv('final_result.csv', sep='\t')
df[(df['Iterative'] == False) & (df['SampleFromAll'] == False)].groupby(['Model', 'C'], as_index=False)['MAP@5', 'MAP@10', 'MAP@15', 'MAP@20', 'MAP@50', 'NDCG',
'Precision@5', 'Precision@10', 'Precision@15', 'Precision@20', 'Precision@50', 'R-Precision', 'Recall@5',
'Recall@10', 'Recall@15', 'Recall@20', 'Recall@50'].agg({'MAP@5': 'mean', 'MAP@10': 'mean', 'MAP@15': 'mean', 
'MAP@20': 'mean', 'MAP@50': 'mean', 'NDCG': 'mean', 'Precision@5': 'mean', 'Precision@10': 'mean', 
'Precision@15': 'mean', 'Precision@20': 'mean', 'Precision@50': 'mean', 'R-Precision': 'mean', 'Recall@5': 'mean',
'Recall@10': 'mean', 'Recall@15': 'mean', 'Recall@20': 'mean', 'Recall@50': 'mean'})

,Model,C,MAP@5,MAP@10,MAP@15,MAP@20,MAP@50,NDCG,Precision@5,Precision@10,Precision@15,Precision@20,Precision@50,R-Precision,Recall@5,Recall@10,Recall@15,Recall@20,Recall@50
0,Greedy,0.0,0.636387,0.599495,0.568758,0.542806,0.439841,0.293907,0.602961,0.539150,0.489134,0.450472,0.314765,0.223278,0.044158,0.078547,0.106479,0.130231,0.226431
1,ThompsonSampling,0.0,0.613557,0.573595,0.542864,0.517744,0.421410,0.285719,0.573165,0.510961,0.464357,0.428661,0.307414,0.218842,0.042046,0.074824,0.101652,0.124709,0.222046
2,UCB,0.5,0.638286,0.597731,0.564335,0.537278,0.433669,0.291819,0.598583,0.531654,0.479213,0.440945,0.311622,0.221447,0.043819,0.077472,0.104625,0.128134,0.224950
3,UCB,1.0,0.626317,0.587666,0.557223,0.531858,0.431091,0.289660,0.586835,0.526677,0.478824,0.440961,0.309928,0.220472,0.043156,0.076987,0.104670,0.128068,0.223323
4,UCB,2.0,0.626614,0.587225,0.555322,0.528814,0.427787,0.288738,0.589291,0.522520,0.472504,0.435701,0.308769,0.219830,0.043310,0.076412,0.103246,0.126687,0.223222


# Research Question 2: Is doing active learning iteratively on active set helping recommender system's general performance?  


Table 5. Greedy Algorithm, Sample from all data

In [5]:
import pandas as pd
df = pd.read_csv('final_result.csv', sep='\t')
df[(df['Model'] == 'Greedy') & (df['SampleFromAll'] == True)].groupby('Iterative', as_index=False)['MAP@5', 'MAP@10', 'MAP@15', 'MAP@20', 'MAP@50', 'NDCG',
'Precision@5', 'Precision@10', 'Precision@15', 'Precision@20', 'Precision@50', 'R-Precision', 'Recall@5',
'Recall@10', 'Recall@15', 'Recall@20', 'Recall@50'].agg({'MAP@5': 'mean', 'MAP@10': 'mean', 'MAP@15': 'mean', 
'MAP@20': 'mean', 'MAP@50': 'mean', 'NDCG': 'mean', 'Precision@5': 'mean', 'Precision@10': 'mean', 
'Precision@15': 'mean', 'Precision@20': 'mean', 'Precision@50': 'mean', 'R-Precision': 'mean', 'Recall@5': 'mean',
'Recall@10': 'mean', 'Recall@15': 'mean', 'Recall@20': 'mean', 'Recall@50': 'mean'})

,Iterative,MAP@5,MAP@10,MAP@15,MAP@20,MAP@50,NDCG,Precision@5,Precision@10,Precision@15,Precision@20,Precision@50,R-Precision,Recall@5,Recall@10,Recall@15,Recall@20,Recall@50
0,False,0.559470,0.516241,0.483123,0.455906,0.349279,0.224324,0.519055,0.446677,0.398068,0.358630,0.230961,0.160961,0.037405,0.063629,0.084639,0.101207,0.163064
1,True,0.567882,0.525302,0.491568,0.463978,0.360013,0.233142,0.525827,0.456693,0.403990,0.367224,0.241811,0.169505,0.037910,0.065046,0.085826,0.103835,0.171665


Table 6. Greedy Algorithm, Sample from only positive data

In [6]:
import pandas as pd
df = pd.read_csv('final_result.csv', sep='\t')
df[(df['Model'] == 'Greedy') & (df['SampleFromAll'] == False)].groupby('Iterative', as_index=False)['MAP@5', 'MAP@10', 'MAP@15', 'MAP@20', 'MAP@50', 'NDCG',
'Precision@5', 'Precision@10', 'Precision@15', 'Precision@20', 'Precision@50', 'R-Precision', 'Recall@5',
'Recall@10', 'Recall@15', 'Recall@20', 'Recall@50'].agg({'MAP@5': 'mean', 'MAP@10': 'mean', 'MAP@15': 'mean', 
'MAP@20': 'mean', 'MAP@50': 'mean', 'NDCG': 'mean', 'Precision@5': 'mean', 'Precision@10': 'mean', 
'Precision@15': 'mean', 'Precision@20': 'mean', 'Precision@50': 'mean', 'R-Precision': 'mean', 'Recall@5': 'mean',
'Recall@10': 'mean', 'Recall@15': 'mean', 'Recall@20': 'mean', 'Recall@50': 'mean'})

,Iterative,MAP@5,MAP@10,MAP@15,MAP@20,MAP@50,NDCG,Precision@5,Precision@10,Precision@15,Precision@20,Precision@50,R-Precision,Recall@5,Recall@10,Recall@15,Recall@20,Recall@50
0,False,0.636387,0.599495,0.568758,0.542806,0.439841,0.293907,0.602961,0.539150,0.489134,0.450472,0.314765,0.223278,0.044158,0.078547,0.106479,0.130231,0.226431
1,True,0.625411,0.584062,0.555060,0.531863,0.440809,0.301108,0.580913,0.522677,0.482016,0.449843,0.329398,0.233702,0.042423,0.075868,0.104571,0.130009,0.237275


Table 7. Thompson Sampling, Sample from all data

In [7]:
import pandas as pd
df = pd.read_csv('final_result.csv', sep='\t')
df[(df['Model'] == 'ThompsonSampling') & (df['SampleFromAll'] == True)].groupby('Iterative', as_index=False)['MAP@5', 'MAP@10', 'MAP@15', 'MAP@20', 'MAP@50', 'NDCG',
'Precision@5', 'Precision@10', 'Precision@15', 'Precision@20', 'Precision@50', 'R-Precision', 'Recall@5',
'Recall@10', 'Recall@15', 'Recall@20', 'Recall@50'].agg({'MAP@5': 'mean', 'MAP@10': 'mean', 'MAP@15': 'mean', 
'MAP@20': 'mean', 'MAP@50': 'mean', 'NDCG': 'mean', 'Precision@5': 'mean', 'Precision@10': 'mean', 
'Precision@15': 'mean', 'Precision@20': 'mean', 'Precision@50': 'mean', 'R-Precision': 'mean', 'Recall@5': 'mean',
'Recall@10': 'mean', 'Recall@15': 'mean', 'Recall@20': 'mean', 'Recall@50': 'mean'})

,Iterative,MAP@5,MAP@10,MAP@15,MAP@20,MAP@50,NDCG,Precision@5,Precision@10,Precision@15,Precision@20,Precision@50,R-Precision,Recall@5,Recall@10,Recall@15,Recall@20,Recall@50
0,False,0.518442,0.474580,0.442955,0.418123,0.329114,0.217527,0.470425,0.407276,0.363066,0.331449,0.230362,0.160909,0.033785,0.057846,0.076937,0.093807,0.163376
1,True,0.561318,0.520612,0.487964,0.460984,0.357428,0.230464,0.521890,0.453512,0.403633,0.366031,0.238929,0.167140,0.037526,0.064626,0.085784,0.103524,0.169395


Table 8. Thompson Sampling, Sample from only positive data

In [8]:
import pandas as pd
df = pd.read_csv('final_result.csv', sep='\t')
df[(df['Model'] == 'ThompsonSampling') & (df['SampleFromAll'] == False)].groupby('Iterative', as_index=False)['MAP@5', 'MAP@10', 'MAP@15', 'MAP@20', 'MAP@50', 'NDCG',
'Precision@5', 'Precision@10', 'Precision@15', 'Precision@20', 'Precision@50', 'R-Precision', 'Recall@5',
'Recall@10', 'Recall@15', 'Recall@20', 'Recall@50'].agg({'MAP@5': 'mean', 'MAP@10': 'mean', 'MAP@15': 'mean', 
'MAP@20': 'mean', 'MAP@50': 'mean', 'NDCG': 'mean', 'Precision@5': 'mean', 'Precision@10': 'mean', 
'Precision@15': 'mean', 'Precision@20': 'mean', 'Precision@50': 'mean', 'R-Precision': 'mean', 'Recall@5': 'mean',
'Recall@10': 'mean', 'Recall@15': 'mean', 'Recall@20': 'mean', 'Recall@50': 'mean'})

,Iterative,MAP@5,MAP@10,MAP@15,MAP@20,MAP@50,NDCG,Precision@5,Precision@10,Precision@15,Precision@20,Precision@50,R-Precision,Recall@5,Recall@10,Recall@15,Recall@20,Recall@50
0,False,0.613557,0.573595,0.542864,0.517744,0.421410,0.285719,0.573165,0.510961,0.464357,0.428661,0.307414,0.218842,0.042046,0.074824,0.101652,0.124709,0.222046
1,True,0.621474,0.582742,0.554001,0.530491,0.439252,0.299699,0.582173,0.522268,0.479895,0.447606,0.327843,0.232349,0.042478,0.075856,0.104130,0.129169,0.236139


Table 9. UCB, C=0.5, Sample from all data

In [9]:
import pandas as pd
df = pd.read_csv('final_result.csv', sep='\t')
df[(df['Model'] == 'UCB') & (df['SampleFromAll'] == True) & (df['C'] == 0.5)].groupby('Iterative', as_index=False)['MAP@5', 'MAP@10', 'MAP@15', 'MAP@20', 'MAP@50', 'NDCG',
'Precision@5', 'Precision@10', 'Precision@15', 'Precision@20', 'Precision@50', 'R-Precision', 'Recall@5',
'Recall@10', 'Recall@15', 'Recall@20', 'Recall@50'].agg({'MAP@5': 'mean', 'MAP@10': 'mean', 'MAP@15': 'mean', 
'MAP@20': 'mean', 'MAP@50': 'mean', 'NDCG': 'mean', 'Precision@5': 'mean', 'Precision@10': 'mean', 
'Precision@15': 'mean', 'Precision@20': 'mean', 'Precision@50': 'mean', 'R-Precision': 'mean', 'Recall@5': 'mean',
'Recall@10': 'mean', 'Recall@15': 'mean', 'Recall@20': 'mean', 'Recall@50': 'mean'})

,Iterative,MAP@5,MAP@10,MAP@15,MAP@20,MAP@50,NDCG,Precision@5,Precision@10,Precision@15,Precision@20,Precision@50,R-Precision,Recall@5,Recall@10,Recall@15,Recall@20,Recall@50
0,False,0.548774,0.502971,0.468229,0.440940,0.341609,0.222113,0.499843,0.430551,0.381365,0.345276,0.230535,0.161268,0.035883,0.061287,0.080923,0.097527,0.163510
1,True,0.568339,0.525290,0.491442,0.464425,0.359649,0.231867,0.531654,0.457402,0.405039,0.369173,0.239748,0.168059,0.038300,0.065245,0.086142,0.104449,0.170173


Table 10. UCB, C=0.5, Sample from only positive data

In [10]:
import pandas as pd
df = pd.read_csv('final_result.csv', sep='\t')
df[(df['Model'] == 'UCB') & (df['SampleFromAll'] == False) & (df['C'] == 0.5)].groupby('Iterative', as_index=False)['MAP@5', 'MAP@10', 'MAP@15', 'MAP@20', 'MAP@50', 'NDCG',
'Precision@5', 'Precision@10', 'Precision@15', 'Precision@20', 'Precision@50', 'R-Precision', 'Recall@5',
'Recall@10', 'Recall@15', 'Recall@20', 'Recall@50'].agg({'MAP@5': 'mean', 'MAP@10': 'mean', 'MAP@15': 'mean', 
'MAP@20': 'mean', 'MAP@50': 'mean', 'NDCG': 'mean', 'Precision@5': 'mean', 'Precision@10': 'mean', 
'Precision@15': 'mean', 'Precision@20': 'mean', 'Precision@50': 'mean', 'R-Precision': 'mean', 'Recall@5': 'mean',
'Recall@10': 'mean', 'Recall@15': 'mean', 'Recall@20': 'mean', 'Recall@50': 'mean'})

,Iterative,MAP@5,MAP@10,MAP@15,MAP@20,MAP@50,NDCG,Precision@5,Precision@10,Precision@15,Precision@20,Precision@50,R-Precision,Recall@5,Recall@10,Recall@15,Recall@20,Recall@50
0,False,0.638286,0.597731,0.564335,0.537278,0.433669,0.291819,0.598583,0.531654,0.479213,0.440945,0.311622,0.221447,0.043819,0.077472,0.104625,0.128134,0.224950
1,True,0.631832,0.589725,0.560081,0.535568,0.440719,0.300095,0.588819,0.527874,0.484199,0.449449,0.326047,0.231266,0.043138,0.076818,0.105220,0.130043,0.234936


Table 11. UCB, C=1.0, Sample from all data

In [11]:
import pandas as pd
df = pd.read_csv('final_result.csv', sep='\t')
df[(df['Model'] == 'UCB') & (df['SampleFromAll'] == True) & (df['C'] == 1.0)].groupby('Iterative', as_index=False)['MAP@5', 'MAP@10', 'MAP@15', 'MAP@20', 'MAP@50', 'NDCG',
'Precision@5', 'Precision@10', 'Precision@15', 'Precision@20', 'Precision@50', 'R-Precision', 'Recall@5',
'Recall@10', 'Recall@15', 'Recall@20', 'Recall@50'].agg({'MAP@5': 'mean', 'MAP@10': 'mean', 'MAP@15': 'mean', 
'MAP@20': 'mean', 'MAP@50': 'mean', 'NDCG': 'mean', 'Precision@5': 'mean', 'Precision@10': 'mean', 
'Precision@15': 'mean', 'Precision@20': 'mean', 'Precision@50': 'mean', 'R-Precision': 'mean', 'Recall@5': 'mean',
'Recall@10': 'mean', 'Recall@15': 'mean', 'Recall@20': 'mean', 'Recall@50': 'mean'})

,Iterative,MAP@5,MAP@10,MAP@15,MAP@20,MAP@50,NDCG,Precision@5,Precision@10,Precision@15,Precision@20,Precision@50,R-Precision,Recall@5,Recall@10,Recall@15,Recall@20,Recall@50
0,False,0.541444,0.498199,0.465248,0.438419,0.339709,0.221170,0.498898,0.430394,0.380997,0.344110,0.230431,0.161174,0.035883,0.061284,0.080895,0.097398,0.163288
1,True,0.569887,0.526123,0.492442,0.465019,0.360414,0.233027,0.526677,0.455874,0.406341,0.368504,0.241493,0.169060,0.037898,0.064948,0.086318,0.104304,0.171330


Table 12. UCB, C=1.0, Sample from only positive data

In [12]:
import pandas as pd
df = pd.read_csv('final_result.csv', sep='\t')
df[(df['Model'] == 'UCB') & (df['SampleFromAll'] == False) & (df['C'] == 1.0)].groupby('Iterative', as_index=False)['MAP@5', 'MAP@10', 'MAP@15', 'MAP@20', 'MAP@50', 'NDCG',
'Precision@5', 'Precision@10', 'Precision@15', 'Precision@20', 'Precision@50', 'R-Precision', 'Recall@5',
'Recall@10', 'Recall@15', 'Recall@20', 'Recall@50'].agg({'MAP@5': 'mean', 'MAP@10': 'mean', 'MAP@15': 'mean', 
'MAP@20': 'mean', 'MAP@50': 'mean', 'NDCG': 'mean', 'Precision@5': 'mean', 'Precision@10': 'mean', 
'Precision@15': 'mean', 'Precision@20': 'mean', 'Precision@50': 'mean', 'R-Precision': 'mean', 'Recall@5': 'mean',
'Recall@10': 'mean', 'Recall@15': 'mean', 'Recall@20': 'mean', 'Recall@50': 'mean'})

,Iterative,MAP@5,MAP@10,MAP@15,MAP@20,MAP@50,NDCG,Precision@5,Precision@10,Precision@15,Precision@20,Precision@50,R-Precision,Recall@5,Recall@10,Recall@15,Recall@20,Recall@50
0,False,0.626317,0.587666,0.557223,0.531858,0.431091,0.289660,0.586835,0.526677,0.478824,0.440961,0.309928,0.220472,0.043156,0.076987,0.104670,0.128068,0.223323
1,True,0.624056,0.583961,0.555210,0.531770,0.440146,0.300145,0.581984,0.522803,0.481491,0.449181,0.327761,0.232511,0.042550,0.075978,0.104674,0.129830,0.236125


Table 13. UCB, C=2.0, Sample from all data

In [13]:
import pandas as pd
df = pd.read_csv('final_result.csv', sep='\t')
df[(df['Model'] == 'UCB') & (df['SampleFromAll'] == True) & (df['C'] == 2.0)].groupby('Iterative', as_index=False)['MAP@5', 'MAP@10', 'MAP@15', 'MAP@20', 'MAP@50', 'NDCG',
'Precision@5', 'Precision@10', 'Precision@15', 'Precision@20', 'Precision@50', 'R-Precision', 'Recall@5',
'Recall@10', 'Recall@15', 'Recall@20', 'Recall@50'].agg({'MAP@5': 'mean', 'MAP@10': 'mean', 'MAP@15': 'mean', 
'MAP@20': 'mean', 'MAP@50': 'mean', 'NDCG': 'mean', 'Precision@5': 'mean', 'Precision@10': 'mean', 
'Precision@15': 'mean', 'Precision@20': 'mean', 'Precision@50': 'mean', 'R-Precision': 'mean', 'Recall@5': 'mean',
'Recall@10': 'mean', 'Recall@15': 'mean', 'Recall@20': 'mean', 'Recall@50': 'mean'})

,Iterative,MAP@5,MAP@10,MAP@15,MAP@20,MAP@50,NDCG,Precision@5,Precision@10,Precision@15,Precision@20,Precision@50,R-Precision,Recall@5,Recall@10,Recall@15,Recall@20,Recall@50
0,False,0.531462,0.484496,0.450016,0.423740,0.331025,0.218124,0.482457,0.411055,0.364556,0.332441,0.229109,0.160291,0.034507,0.058493,0.077485,0.094213,0.162616
1,True,0.573098,0.528425,0.493979,0.466115,0.360171,0.232875,0.531528,0.457291,0.405879,0.367559,0.240970,0.168617,0.038263,0.065235,0.086196,0.103964,0.170792


Table 14. UCB, C=2.0, Sample from only positive data

In [14]:
import pandas as pd
df = pd.read_csv('final_result.csv', sep='\t')
df[(df['Model'] == 'UCB') & (df['SampleFromAll'] == False) & (df['C'] == 2.0)].groupby('Iterative', as_index=False)['MAP@5', 'MAP@10', 'MAP@15', 'MAP@20', 'MAP@50', 'NDCG',
'Precision@5', 'Precision@10', 'Precision@15', 'Precision@20', 'Precision@50', 'R-Precision', 'Recall@5',
'Recall@10', 'Recall@15', 'Recall@20', 'Recall@50'].agg({'MAP@5': 'mean', 'MAP@10': 'mean', 'MAP@15': 'mean', 
'MAP@20': 'mean', 'MAP@50': 'mean', 'NDCG': 'mean', 'Precision@5': 'mean', 'Precision@10': 'mean', 
'Precision@15': 'mean', 'Precision@20': 'mean', 'Precision@50': 'mean', 'R-Precision': 'mean', 'Recall@5': 'mean',
'Recall@10': 'mean', 'Recall@15': 'mean', 'Recall@20': 'mean', 'Recall@50': 'mean'})

,Iterative,MAP@5,MAP@10,MAP@15,MAP@20,MAP@50,NDCG,Precision@5,Precision@10,Precision@15,Precision@20,Precision@50,R-Precision,Recall@5,Recall@10,Recall@15,Recall@20,Recall@50
0,False,0.626614,0.587225,0.555322,0.528814,0.427787,0.288738,0.589291,0.522520,0.472504,0.435701,0.308769,0.219830,0.043310,0.076412,0.103246,0.126687,0.223222
1,True,0.625411,0.585460,0.556253,0.532480,0.440413,0.299984,0.582488,0.524126,0.482058,0.448583,0.327515,0.232289,0.042706,0.076199,0.104748,0.129541,0.235747


# Conclusion

From tables 1-4, it is apparent that UCB outperforms greedy algorithm in general and Thompson sampling has the worst general performance.

From tables 5-14, we can observe that recommendation general performance improves when the model actively learns from all data iteratively. However, the general performance doesn't really increase when the model learns from positive data only. Based on those observations, we can conclude that model iteratively learns from data only helps to boost general performance when data is limited (learns from all data). If the model sees enough data (sample from positive data only), one-shot from active set is enough. 